# Ejercicio : Clasificación con scikit-learn/pyspark.ml.classification

`scikit-learn` (o `sklearn`) es una librería que reúne muchas herramientas para realizar Minería de Datos y _Aprendizaje de Máquinas_. Permite hacer clasificación, clustering, entre otras. Además, incluye varios datasets para aprender a usar la librería.

En este Ejercicio se refuerz los conceptos de _aprendizaje supervisado_ y a mostrar cómo usar `sklearn` y `ml.classification` para entrenar nuestro primer clasificador.

Puedes ejecutar cada una de las celdas de código haciendo click en ellas y presionando `Shift + Enter`. 

También puedes editar cualquiera de estas celdas. Las celdas no son independientes. Es decir, sí importa el orden en el que las ejecutes, y cualquier cambio que hagas se reflejará en las celdas que ejecutes después.

---

Cargamos el Iris Dataset que viene en `sklearn`. El dataset contiene 

In [ ]:
from sklearn.datasets import load_iris

iris = load_iris()

print("Atributos:", iris.feature_names)
print()
print("5 primeras filas:")
print(iris.data[0:5])
print(iris.target[0:5])

Los atributos del dataset son el largo y el ancho del pétalo y sépalo de cada flor.

<img src="https://sebastianraschka.com/images/blog/2014/intro_supervised_learning/iris_petal_sepal_1.png" alt="" style="width: 1000px;"/>


Las están dadas por el campo `target`, y los distintos tipos de `target` en el campo `target_names`:

In [ ]:
print(iris.data.shape)
print(iris.target.shape)
print("target_names:", iris.target_names)
print()
print("Valores de la columna")
print(iris.target)

Vemos que los primeros valores de la columna `target` son ceros en vez del nombre de la especie. Lo que se usa comúnmente es _mapear_ (asignar) números a variables categóricas. En este caso, el 0 corresponde a la primera especie en  `target_names`, es decir, a _iris setosa_. El 1 corresponde a _iris versicolor_ y el 2 a _iris virginica_.

Los datos que vienen en `sklearn` ya están listos para ser usados con los métodos de la librería. Si hubiésemos recibido los datos como una tabla, éstos se verían más o menos así:

| sepal length (cm) | sepal width (cm) | petal length (cm) | petal width (cm) | target          |
|-------------------|------------------|-------------------|------------------|-----------------|
| 5.1               | 3.5              | 1.4               | 0.2              | iris-setosa     |
| 4.9               | 3                | 1.4               | 0.2              | iris-setosa     |
| ...               | ...              | ...               | ...              | ...             |
| 5                 | 2                | 3.5               | 1                | iris-versicolor |
| 5.9               | 3                | 5.1               | 1.8              | iris-virginica  |


Una tarea que se nos podría plantear sería determinar, dados los atributos de una flor, cuál es la especie a la que corresponde. Por ejemplo, ¿a cuál especie corresponde la flor con los siguientes atributos?

| sepal length (cm) | sepal width (cm) | petal length (cm) | petal width (cm) | target |
|-------------------|------------------|-------------------|------------------|--------|
| 4.8               | 3                | 1.4               | 0.1              | ???    |

Teniendo los datos cargados vamos a crear la instancia de spark, utilizando `findspark`, donde podemos comprobar la ruta de donde se carga el spark con `findspark.find()`.

In [ ]:
import findspark
findspark.init("C:\Spark\spark-3.0.0-preview2-bin-hadoop2.7")

import pyspark
sc = pyspark.SparkContext(appName="iris")

from pyspark.sql import SQLContext 
sq = SQLContext(sc)
findspark.find()

Aunque no es necesario cargar en RDD los datos, lo hacemos para mostrar como podemos cambiar un tipo `pandas` a un tipo `rdd`

In [ ]:
rdd = sc.parallelize(iris.data)
rdd2 = sc.parallelize(iris.target)
print(rdd.take(2))
print(rdd2.take(2))

Creamos un dataframe en base a los rdd cargados, utilizamos la componente `Row`

In [ ]:
from pyspark.sql import Row
iris_data_df = rdd.map(lambda x: \
               Row(sepalo_largo=float(x[0]), sepalo_ancho=float(x[1]), petalo_largo=float(x[2]), petalo_ancho=float(x[3]))).toDF()

iris_target_df = rdd2.map(lambda y: \
               Row(label=int(y))).toDF()

Vemos la distribución de los `target`, para ver cuantas de cada planta hay en este estudio.
Ahora necesitamos unir los dataset de `data` y `target` ejecutar el clasificador, entonces utilizamos la componente `pandas` para `concatenarlos` (`pandas.concat()`)

In [ ]:
import pandas as pd
datos = iris_target_df.toPandas()
datos.loc[:,'Total'] = 0
print(datos.groupby(['label'],as_index=False).agg({"Total":"count"}))

dataset= pd.concat([iris_data_df.toPandas(), iris_target_df.toPandas()], axis=1,)
print(dataset)

Antes de ejecutar el calsificados, debemos crear `dataframe` en pyspark con `sq.createDataFrame(dataset)`, donde `sq` corresponde al `SQLContext(sc)`.

In [ ]:
data_total = sq.createDataFrame(dataset)
print(data_total.count())


Preparamos la data para lograr entrenar, esto porque la componente `DecisionTreeClassifier` solicita:
1.- Una entrada de `features` que corresponde a un vector con todas las características que se quiere entrenar.
2.- Una entrada de `label` que corresponde a un vector de `target` asociados al entrenamiento.

Entonces,
    para lograr el vector de `features` utilizamos la componente `VectorAssembler` para el set de entrenamiento.

In [ ]:
from pyspark.ml.feature import StringIndexer, VectorAssembler
data_input = ["sepalo_largo", "sepalo_ancho", "petalo_largo", "petalo_ancho"]
#target_input =["resultado"]
assemblerInputs = data_input# + target_input
vector_features = VectorAssembler(inputCols=assemblerInputs, outputCol="features")
total_V = vector_features.transform(data_total)
print(total_V)
print(data_total.columns)

## Nuestro primer clasificador

Vamos a usar un _árbol de decisión_ como nuestro primer clasificador. Un árbol de decisión para este problema puede verse como el de la siguiente imagen:

![](https://sebastianraschka.com/images/blog/2014/intro_supervised_learning/decision_tree_1.png)

Nota que podemos mirar cualquiera de los atributos primero, o no usar algún otro atributo, por ejemplo:

![](https://www.ibm.com/developerworks/library/ba-predictive-analytics2/fig06.gif)

En el último caso no usamos el `petal width` como atributo para el árbol. Y así, podemos tener muchos árboles distintos.

El proceso de _entrenar un clasificador_ corresponde al proceso de —en este caso— encontrar las reglas del árbol que _mejor se adapten a nuestros datos_. Llamaremos al árbol resultante el _**modelo**_.

In [ ]:
from pyspark.ml.classification import DecisionTreeClassifier

# Create initial Decision Tree Model
dt = DecisionTreeClassifier(labelCol="label", featuresCol="features", maxDepth=3)
dtModel = dt.fit(total_V)
print("numNodes = ", dtModel.numNodes)
print("depth = ", dtModel.depth)
print(dtModel.toDebugString)

# ¿Cómo evaluamos nuestro modelo?¶

¿Cómo sabemos qué tan bien le fue? Es decir, ¿logró aprender desde los datos cuáles eran las mejores reglas?

Una forma de ver esto es usando el modelo para clasificar nuevas instancias de los datos.

Sin embargo, no tenemos nuevas instancias, ya que entrenamos el clasificador con todos los datos disponibles. Si evaluamos nuestro clasificador con los datos de entrenamiento (es decir, los datos que usamos para entrenar el clasificador y generar un modelo), vamos a tener resultados sobre-optimistas, ya que el clasificador usó esos mismos datos para entrenar. Es como si fueras a dar una prueba y usaras la misma prueba con las respuestas para estudiar.

Esto también nos entrega una pista sobre qué significa que un clasificador aprenda de los datos. Para que un modelo se considere bueno_, no basta con que clasifique correctamente los datos que usó para entrenar, sino que debe clasificar correctamente datos que _no ha visto antes. Esto es a lo que se llama la capacidad de generalización del modelo.

Vamos a definir un par de conceptos antes de continuar:

El conjunto de datos de entrenamiento, o training set, es el conjunto de datos que le damos al clasificador para que pueda encontrar las reglas o parámetros óptimos que le permitan predecir la clase de estos datos.

El conjunto de datos de prueba, o test set, es el conjunto de datos sobre el cual vamos a evaluar el rendimiento de nuestro modelo. Estos datos se eligen antes de cualquier modificación o limpieza del dataset, y sólo se usan para evaluar el modelo entrenado.

(Una vez seguros de que nuestro modelo funciona bien y queremos usarlo "en producción", podemos entrenar con todos los datos disponibles. No antes)

El último punto es muy importante. Si por ejemplo, normalizamos los datos primero, y después separamos en training y test sets, estaremos "contaminando" nuestros datos de entrenamiento, dándoles información del test set y en cierta forma "haciendo trampa", afectando la capacidad de generalización del modelo resultante.

Holdout
Ahora vamos a tomar una muestra de los datos y separarlos en training set y test set, respectivamente. ¿Cómo determinamos esta muestra?

Antes de ejecutar el calsificados, debemos separar nuestro set de datos en datos de entrenamiento y datos de testing, para ello volvemos a crear dataframe en pyspark con sq.createDataFrame(dataset), donde sq corresponde al SQLContext(sc).

Aplicamos training_total.randomSplit(), considerando para nuestro entrenamiento un 70% de datos de entrenamiento y 30% de datos de testing, que podemos ir modificando en acuerdo a aumentar el accuracy del modelo.

In [ ]:
training_total = sq.createDataFrame(dataset)
(trainingData, testData) = training_total.randomSplit([0.67, 0.33], seed=100)
print(trainingData.count())
print(testData.count())
print(testData.take(10))

Ahora ejecutamos sólo con data de `entrenamiento`

In [ ]:
from pyspark.ml.feature import StringIndexer, VectorAssembler
data_input = ["sepalo_largo", "sepalo_ancho", "petalo_largo", "petalo_ancho"]
#target_input =["resultado"]
assemblerInputs = data_input# + target_input
vector_featuresT = VectorAssembler(inputCols=assemblerInputs, outputCol="features")
train_V = vector_featuresT.transform(trainingData)
print(train_V)
print(training_total.columns)
test_V = vector_featuresT.transform(testData)

Ejecutamos el clasificador:

In [ ]:
from pyspark.ml.classification import DecisionTreeClassifier

# Create initial Decision Tree Model
dt_T = DecisionTreeClassifier(labelCol="label", featuresCol="features", maxDepth=3)
dtModel_T = dt.fit(train_V)
print("numNodes = ", dtModel_T.numNodes)
print("depth = ", dtModel_T.depth)
print(dtModel_T.toDebugString)

Para evaluar, predecimos usando las observaciones en el `test set` y contrastamos el resultado con el `target` correcto.

In [ ]:
predictions = dtModel.transform(test_V)
predictions.select('sepalo_largo', 'sepalo_ancho', 'petalo_largo', 'petalo_ancho','label', 'prediction', 'probability').show(10)

In [ ]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

# Select (prediction, true label) and compute test error
evaluator = MulticlassClassificationEvaluator(
    labelCol="label", predictionCol="prediction", metricName="accuracy")
accuracy = evaluator.evaluate(predictions)
print("Test Error = %g" % (1.0 - accuracy))
print("Precisiòn = %g" % (accuracy*100))
evaluator.evaluate(predictions)

Vemos que el _accuracy_ es de un 97% aprox.. Esto significa que clasificó correctamente el 97% de los datos en `X_test`.

Surgen dos preguntas a partir de esto:

1. ¿Tuvimos suerte? Es decir, si hubiésemos elegido otra partición train/test, ¿obtendríamos resultados diferentes?
2. ¿Qué pasa si las clases están desbalanceadas? ¿Cómo afecta al accuracy si tenemos, por ejemplo, 99% de una clase y 1% de otra?


### Cross-Validation

_¿Tuvimos suerte? Es decir, si hubiésemos elegido otra partición train/test, ¿obtendríamos resultados diferentes?_

Cross-validation nos ayuda a disminuir el efecto del azar (pregunta 1). Por ejemplo, observa qué pasa si cambiamos la semilla aleatoria:

In [ ]:
from pyspark.ml.feature import StringIndexer, VectorAssembler
data_inputCV = ["sepalo_largo", "sepalo_ancho", "petalo_largo", "petalo_ancho"]
#target_input =["resultado"]
assemblerInputsCV = data_inputCV# + target_input
vector_featuresCV = VectorAssembler(inputCols=assemblerInputsCV, outputCol="features")
train_CV = vector_featuresCV.transform(trainingData)
print(train_CV)
print(training_total.columns)
test_CV = vector_featuresCV.transform(testData)

In [ ]:
from pyspark.ml.classification import DecisionTreeClassifier

# Create initial Decision Tree Model
dt_CV = DecisionTreeClassifier(labelCol="label", featuresCol="features", maxDepth=3)
dtModel_CV = dt_CV.fit(train_CV)
print("numNodes = ", dtModel_CV.numNodes)
print("depth = ", dtModel_CV.depth)
print(dtModel_CV.toDebugString)

#train_CV divide en 
#total 100 datos
#train_CV de entrenamiento: del 1 al 80
#train_CV de validación: del 81 al 100

Para disminuir el efecto, _cross-validation_ particiona los datos en $k$ partes iguales, entrena con $k-1$ partes, evalúa en la $k$-ésima, guarda el resultado, y vuelve a repetir el proceso con otras $k-1$ partes hasta haber recorrido todas las partes. Este proceso se llama $k$-fold cross-validation.

Observa que esto implica que el clasificador se entrenará $k$ veces, lo cual puede ser costoso dependiendo del clasificador y de la cantidad de datos.

![](https://upload.wikimedia.org/wikipedia/commons/1/1c/K-fold_cross_validation_EN.jpg)

In [ ]:
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
paramGrid = (ParamGridBuilder()
             .addGrid(dt.maxDepth, [3, 6, 9, 12])
             .addGrid(dt.maxBins, [7, 40, 80, 100])
             .build())

evaluatorCV = MulticlassClassificationEvaluator(
    labelCol="label", predictionCol="prediction", metricName="accuracy")
cv = CrossValidator(estimator=dt_CV, estimatorParamMaps=paramGrid, evaluator=evaluatorCV, numFolds=5)
cvModel = cv.fit(train_CV)
print("numNodes = ", cvModel.bestModel.numNodes)
print("depth = ", cvModel.bestModel.depth)

    

In [ ]:
predictionsCV = cvModel.transform(test_CV)
print(evaluatorCV.evaluate(predictionsCV))
predictionsCV.select('sepalo_largo', 'sepalo_ancho', 'petalo_largo', 'petalo_ancho','label', 'prediction', 'probability').show(10)

In [ ]:
accuracyCV = evaluatorCV.evaluate(predictionsCV)
print("Test Error = %g" % (1.0 - accuracyCV))
print("Precisiòn = %g" % (accuracyCV*100))
evaluatorCV.evaluate(predictionsCV)


## Matriz de confusión, Precision y Recall

La _matriz de confusión_ nos permite observar los errores del clasificador:

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import itertools

def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')



In [ ]:
class_temp = predictionsCV.select("label").groupBy("label")\
                        .count().sort('count', ascending=False).toPandas()
class_temp = class_temp["label"].values.tolist()
class_names = map(int, class_temp)
# # # print(class_name)
class_names

In [ ]:
from sklearn.metrics import confusion_matrix
y_true = predictionsCV.select("label")
y_true = y_true.toPandas()

y_pred = predictionsCV.select("prediction")
y_pred = y_pred.toPandas()

cnf_matrix = confusion_matrix(y_true, y_pred)


La matriz se interpeta de la siguiente forma:

| iris-setosa  | iris-versicolor  | iris-virginica  | ← clasificado como / clase real ↓ |
|:----:|:----:|:----:|--------------------:|
| 12 | 0  |   |              **iris-setosa** |
| 0  | 17  | 1  |              **iris-versicolor** |
| 0  | 2 | 18 |              **iris-virginica** |

Por cada clase, podemos determinar el tipo de error que el modelo hace.

- **Verdaderos Positivos (TP)**: el dato X es de la clase C, y el modelo clasifica X como C.
- **Verdaderos Negativos (TN)**: el dato X no es de la clase C, y el modelo clasifica X como algo que no es C.
- **Falsos Positivos (FP)**: el dato X no es de la clase C, pero el modelo clasifica a X como C.
- **Falsos Negativos (FN)**: el dato X es de la clase C, pero el modelo clasifica a X como algo que no es C.

A partir de estas medidas, definimos dos medidas nuevas para una clase, _precision_ y _recall_:

$$Precision = \frac{TP}{TP + FP}$$

$$Recall = \frac{TP}{TP+FN}$$

![](https://wikimedia.org/api/rest_v1/media/math/render/svg/e2e427ec6dcf2d7882c3bbdc659a8204cba59dcc)

Nota que estas medidas son para una clase en particular. La medida para todo el dataset puede ser el promedio de la medida para cada clase.

Ver más en https://en.wikipedia.org/wiki/Precision_and_recall

In [ ]:
plt.figure()
plot_confusion_matrix(cnf_matrix, classes=list(class_names),
                      title='Confusion matrix, without normalization')
plt.show()

In [ ]:
# Plot normalized confusion matrix
plt.figure()
plot_confusion_matrix(cnf_matrix, classes=list(class_names), normalize=True,
                      title='Normalized confusion matrix')

plt.show()

# En conclusión

El flujo usual a la hora de entrenar un clasificador es el siguiente:

1. Tener datos. Verificar la fuente de los datos, la existencia de sesgos (sesgo de selección, sesgo del superviviente, sesgos sociodemográficos, etc.).
2. Separar datos en train y test set.
3. Realizar exploración y limpieza de datos en ambos sets, de manera independiente.
4. Elegir clasificadores apropiados para el dominio del problema (próxima clase de cátedra)
5. Determinar métricas de entrenamiento usando cross-validation, si es posible (más de esto en el lab de mañana).
6. Evaluar en el test set.
7. Usar todos los datos para entrenar el modelo que irá "a producción"

# Referencias

1. Documentación de scikit-learn. http://scikit-learn.org/stable/index.html
2. Precision y Recall. https://en.wikipedia.org/wiki/Precision_and_recall
3. Machine Learning 101 (Google). https://docs.google.com/presentation/d/1kSuQyW5DTnkVaZEjGYCkfOxvzCqGEFzWBy4e9Uedd9k/preview?imm_mid=0f9b7e&cmp=em-data-na-na-newsltr_20171213#slide=id.g168a3288f7_0_58
4. WEKA (un programa visual con clasificadores y otras herramientas para ML). https://www.cs.waikato.ac.nz/ml/weka/
5. Curso de Data Mining con WEKA. https://www.cs.waikato.ac.nz/ml/weka/mooc/dataminingwithweka/

# Random Forest

En Random Forest se ejecutan varios algoritmos de árbol de decisiones en lugar de uno solo. Para clasificar un nuevo objeto basado en atributos, cada árbol de decisión da una clasificación y finalmente la decisión con mayor “votos” es la predicción del algoritmo.

![](https://iartificial.net/wp-content/uploads/2019/06/Random-Forest-Bagging.png)

In [ ]:
from pyspark.ml.feature import StringIndexer, VectorAssembler
data_inputRF = ["sepalo_largo", "sepalo_ancho", "petalo_largo", "petalo_ancho"]
#target_input =["resultado"]
assemblerInputsRF = data_inputRF# + target_input
vector_featuresRF = VectorAssembler(inputCols=assemblerInputsRF, outputCol="features")
train_RF = vector_featuresRF.transform(trainingData)
print(train_RF)
print(training_total.columns)
test_RF = vector_featuresRF.transform(testData)

In [ ]:
from pyspark.ml.classification import RandomForestClassifier
rf = RandomForestClassifier(featuresCol = 'features', labelCol = 'label', numTrees=10)
rfModel = rf.fit(train_RF)
predictionsRF = rfModel.transform(test_RF)
predictionsRF.select('sepalo_largo', 'sepalo_ancho', 'petalo_largo', 'petalo_ancho','label', 'prediction', 'probability').show(10)

In [ ]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

evaluatorRF = MulticlassClassificationEvaluator(
    labelCol="label", predictionCol="prediction", metricName="accuracy")
accuracyRF = evaluatorRF.evaluate(predictionsRF)
print("Test Error = %g" % (1.0 - accuracyRF))
print("Precisiòn = %g" % (accuracyRF*100))
evaluatorRF.evaluate(predictionsRF)

# Clasificadores binarios

Pasamos a repasar clasificadores binarios, que se pueden utilizar. Pero como en el set tenemos multiclases, el modelo debe pasar por la reduccción "One-vs-Al" componente "OneVsRest" que en base a un clasificador crea una binaria clasificación.

# Gradient-Boosted tree Classifier

Los clasificadores de aumento de gradiente son un grupo de algoritmos de aprendizaje automático que combinan muchos modelos de aprendizaje débiles para crear un modelo predictivo sólido. Los árboles de decisión generalmente se usan al aumentar el gradiente.

![](https://2.bp.blogspot.com/-Dx97g4KSWGw/Ww0Aa8RyGUI/AAAAAAAABnc/hdEpxgQ-XforLeICrdwYisCYNJN8KuLDQCLcBGAs/s400/gradient-boosting.png)

In [ ]:
from pyspark.ml.feature import StringIndexer, VectorAssembler
data_inputGB = ["sepalo_largo", "sepalo_ancho", "petalo_largo", "petalo_ancho"]
#target_input =["resultado"]
assemblerInputsGB = data_inputGB# + target_input
vector_featuresGB = VectorAssembler(inputCols=assemblerInputsGB, outputCol="features")
train_GB = vector_featuresGB.transform(trainingData)
print(train_GB)
print(training_total.columns)
test_GB = vector_featuresGB.transform(testData)

In [ ]:
print(train_GB.show())

In [ ]:
from pyspark.ml.classification import GBTClassifier, OneVsRest
gbt = GBTClassifier(maxIter=5, maxDepth=2, featuresCol='features', labelCol="label", seed=100)
#gbt = GBTClassifier(maxIter=5, seed=100)
ovrgbt = OneVsRest(classifier=gbt)
print(ovrgbt.getParam)
gbtModel = ovrgbt.fit(train_GB)
predictionsgbt = gbtModel.transform(test_GB)
predictionsgbt.select('sepalo_largo', 'sepalo_ancho', 'petalo_largo', 'petalo_ancho','label', 'prediction').show(10)

In [ ]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

evaluatorgbt = MulticlassClassificationEvaluator(
    labelCol="label", predictionCol="prediction", metricName="accuracy")
accuracygbt = evaluatorgbt.evaluate(predictionsgbt)
print("Test Error = %g" % (1.0 - accuracygbt))
print("Precisiòn = %g" % (accuracygbt*100))
evaluatorgbt.evaluate(predictionsgbt)

# Support Vector Machine

Una máquina de vectores de soporte (SVM) es un modelo de aprendizaje automático supervisado que utiliza algoritmos de clasificación para problemas de clasificación de dos grupos. 

El objetivo del algoritmo de máquina de vectores de soporte es encontrar un hiperplano en un espacio N-dimensional (N - el número de características) que clasifica claramente los puntos de datos.

![](https://iartificial.net/wp-content/uploads/2019/04/Clasificacion-SVM-1024x582.png)

In [ ]:
from pyspark.ml.feature import StringIndexer, VectorAssembler
data_inputSVM = ["sepalo_largo", "sepalo_ancho", "petalo_largo", "petalo_ancho"]
#target_input =["resultado"]
assemblerInputsSVM = data_inputSVM# + target_input
vector_featuresSVM = VectorAssembler(inputCols=assemblerInputsSVM, outputCol="features")
train_SVM = vector_featuresSVM.transform(trainingData)
print(train_SVM)
print(training_total.columns)
test_SVM = vector_featuresSVM.transform(testData)

In [ ]:
from pyspark.ml.classification import LinearSVC, OneVsRest
lsvc = LinearSVC(maxIter=10, regParam=0.1)
ovrSVM = OneVsRest(classifier=lsvc)
lsvcModel = ovrSVM.fit(train_SVM)
predictionslsvc = lsvcModel.transform(test_SVM)
predictionslsvc.select('sepalo_largo', 'sepalo_ancho', 'petalo_largo', 'petalo_ancho','label', 'prediction').show(10)

In [ ]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

evaluatorSVM = MulticlassClassificationEvaluator(
    labelCol="label", predictionCol="prediction", metricName="accuracy")
accuracySVM = evaluatorSVM.evaluate(predictionslsvc)
print("Test Error = %g" % (1.0 - accuracySVM))
print("Precisiòn = %g" % (accuracySVM*100))
evaluatorSVM.evaluate(predictionslsvc)

In [ ]:
sc.stop()